### Toronto Data Wrangling

#### Import necessary libraries

In [2]:
import pandas as pd

#### Code

In [3]:
#Read file
dfs =  pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#Get relevant dataframe
df = dfs[0]
#Remove not assigned
df = df[df['Borough'] != 'Not assigned']
#Group the data
df_grouped = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df_grouped['Neighborhood']= df_grouped['Neighborhood'].str.replace('/',',')
#Print head
df_grouped.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#Data Shape 
df_grouped.shape

(103, 3)

In [16]:
#Rename the table
df_grouped = df_grouped.rename(columns = {'Postal code': 'Postal Code'})

#Read csv and join data
geo_data = pd.read_csv(r'H:\Python\Coursera\Data Analysis\Geospatial_Coordinates.csv')
df_geo = pd.merge(df_grouped, geo_data, on = 'Postal Code', how = 'left')
df_geo.head()

df_geo[df_geo['Postal Code']=='M4M']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [14]:
import folium



map = folium.Map(location=[43.761631,-79.520999], zoom_start=11)
folium.Marker([43.761631,-79.520999], popup='Downsview').add_to(map)
map

In [18]:
import folium



map = folium.Map(location=[43.659526,-79.340923], zoom_start=11)
folium.Marker([43.659526,-79.340923], popup='Studio District').add_to(map)
map

In [23]:
import folium

locations = df_geo[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()



map = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_geo['Postal Code'][point]).add_to(map)

map




In [70]:

import foursquare as fs
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner 
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


In [71]:
CLIENT_ID = 'B2A2O12OGUVOCNDPQGCJUMU343MXVSBXRWTOC3DUPJOB0FM1'
CLIENT_SECRET = 'YZDYE4QW1Z01QJXI0QGTDOD0SYV2JR24KBLA5NHR2NZLBMNQ'
LIMIT = 100
VERSION = '20200412'
client = fs.Foursquare(client_id= CLIENT_ID, client_secret = CLIENT_SECRET)

In [72]:



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [73]:
neighborhood_name = df_geo.loc[0,'Neighborhood']
neighborhood_latitude = df_geo.loc[0,'Latitude']
neighborhood_longitude = df_geo.loc[0,'Longitude']
LIMIT = 100
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)


C:\Users\miguel.torres\.conda\envs\spyder\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [74]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.802008,-79.198080
1,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
2,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
3,Harvey's,Restaurant,43.800020,-79.198307
4,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607


In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
neighborhood_name = df_geo.loc[0,'Neighborhood']
neighborhood_latitude = df_geo.loc[0,'Latitude']
neighborhood_longitude = df_geo.loc[0,'Longitude']


list_venues=getNearbyVenues(names=df_geo['Neighborhood'], latitudes=df_geo['Latitude'],
                    longitudes=df_geo['Longitude'])
list_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern , Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Malvern , Rouge",43.806686,-79.194353,T Hamilton & Son Roofing Inc,43.807985,-79.198194,Construction & Landscaping
3,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...
2141,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437,Sunny Foodmart,43.741840,-79.590561,Grocery Store
2142,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437,McDonald's,43.741757,-79.584230,Fast Food Restaurant
2143,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437,Pizza Nova,43.736761,-79.589817,Pizza Place
2144,Northwest,43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location


In [85]:
neighborhood_venues_grouped = list_venues.groupby('Neighborhood').count()
neighborhood_venues_grouped.sort_values('Venue', ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"First Canadian Place , Underground city",100,100,100,100,100,100
"Richmond , Adelaide , King",100,100,100,100,100,100
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
"Toronto Dominion Centre , Design Exchange",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
...,...,...,...,...,...,...
"West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale",1,1,1,1,1,1
Roselawn,1,1,1,1,1,1
Weston,1,1,1,1,1,1


In [86]:

# one hot encoding
toronto_onehot = pd.get_dummies(list_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = list_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))

toronto_onehot = toronto_onehot.reindex(columns= cols)
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
toronto_grouped.describe()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,0.001754,0.000140,0.001468,0.000658,0.000658,0.001316,0.001974,0.001316,0.011251,0.000746,...,0.000316,0.002669,0.000552,0.001404,0.004938,0.000501,0.001473,0.000810,0.003225,0.004558
std,0.017100,0.001368,0.010116,0.006412,0.006412,0.012825,0.019237,0.012825,0.053138,0.004641,...,0.001758,0.008152,0.003323,0.009924,0.023749,0.004886,0.004651,0.007892,0.025848,0.012466
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.166667,0.013333,0.076923,0.062500,0.062500,0.125000,0.187500,0.125000,0.500000,0.038462,...,0.010000,0.046875,0.027027,0.083333,0.200000,0.047619,0.025000,0.076923,0.250000,0.055556


In [92]:
toronto_grouped_sum = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped_sum.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood , Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor , Wilson Heights , Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park , Lawrence Manor East",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [93]:
toronto_grouped_sum.describe()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,0.021053,0.010526,0.021053,0.010526,0.010526,0.021053,0.031579,0.021053,0.294737,0.031579,...,0.031579,0.168421,0.031579,0.021053,0.105263,0.010526,0.105263,0.010526,0.052632,0.157895
std,0.205196,0.102598,0.144321,0.102598,0.102598,0.205196,0.307794,0.205196,0.756162,0.175804,...,0.175804,0.476080,0.175804,0.144321,0.424607,0.102598,0.308520,0.102598,0.267710,0.420632
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,4.000000,1.000000,...,1.000000,3.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,2.000000,2.000000


In [108]:
toronto_grouped_sum.iloc[:,1:].sum(axis=0).sort_values()

Frozen Yogurt Shop           1
German Restaurant            1
Taiwanese Restaurant         1
Garden Center                1
Fruit & Vegetable Store      1
                          ... 
Italian Restaurant          50
Park                        52
Restaurant                  74
Café                        98
Coffee Shop                180
Length: 261, dtype: int64